### DataJam 2025 | Data Cleaning Workshop

-------

Welcome to the first DataJam 2025 workshop - Data Cleaning with Python!

To participate in this workshop, you will first need to download our dataset "NFL Play by Play 2009-2017 (v4).csv", sourced from Kaggle (Max Horowitz). If you've already downloaded the data via the "DataJam 2025 Preparation" email, you are set! If not, you can quickly download the dataset here: [Direct Link](https://drive.google.com/file/d/1EW8W8l5tTxFIXCaUdP9iixPzVm4vs70a/view?usp=sharing)

---

In [1]:
# installing libraries
%pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.


### 1. Importing our data

For this workshop (and for the data visualization workshop), we will be using NFL Play-by-Play data. If you don't know much about NFL, that's okay, as we will just be looking at the numbers.

Each row of our dataset pertains to a play in a game. On average, one NFL game has about 153 plays (NFL.com).

In [5]:
# Essential libraries 
import pandas as pd # 'pd' is a typical alias for pandas
import numpy as np  # 'np' is a typical alias for numpy
np.random.seed(0)  # For reproducibility

Now, using pandas, we can import our .csv file.

In [ ]:
# importing the dataset
path = 'data/NFL Play by Play 2009-2017 (v4).csv' # change this depending on where your dataset is located
data = pd.read_csv(path)
alt_data = data.copy() # for later useage
data.head() # returns the first 5 rows of the dataset

C:\Users\Daniel\AppData\Local\Temp\ipykernel_18428\3374505772.py:2: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/NFL Play by Play 2009-2017 (v4).csv')


,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2009-09-10,2009091000,1,1,NaN,15:00,15,3600.0,0.0,TEN,...,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,...,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,...,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,...,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,...,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


After running data.head(), we can already see lots of 'NaN', or missing data values in our dataset. Rather than looking through the whole dataset, we can use "data.isnull().sum()" to get the total number of NaNs per column (feature).

In [8]:
missing_values = data.isnull().sum()
missing_values

Date             0
GameID           0
Drive            0
qtr              0
down         61154
             ...  
Win_Prob     25009
WPA           5541
airWPA      248501
yacWPA      248762
Season           0
Length: 102, dtype: int64

We have 102 features, so our missing_values table is automatically truncated, however, we can see that there are quite a lot of null values for some features. We can display this is a percentage as well, to get a better understanding of how much data we are missing. 

In [10]:
total_cells = np.prod(data.shape)
total_missing = missing_values.sum()

print(f"Total missing values: {total_missing}")
print(f"Percentage of missing values: {round((total_missing/total_cells) * 100, 2)}%")

Total missing values: 11505187
Percentage of missing values: 27.67%


**Discussion Question:** Why are there so many missing values?

**Answer**: *We need to investigate into the data.* For the purpose of this workshop, we have done this for you. The steps taken are:

1. identify problematic features (most NaN values)
2. interpret problematic features
3. handle problematic features

In [13]:
# identify problematic features
highest_nan = missing_values.sort_values(ascending=False).head(5)
highest_nan

DefTwoPoint         407664
BlockingPlayer      407571
TwoPointConv        407083
ChalReplayResult    404286
RecFumbPlayer       403315
dtype: int64

### Interpret problematic features

According to our dataset, these features are described as such:

- `DefTwoPoint`: "The defender who attempted / made a two-point conversion (i.e., after a touchdown when a team goes for 2 instead of an extra point)"

- `BlockingPlayer`: "The player who executed a block on the specific play (likely for a kicker/returner or special teams, or for a two-point try)"

- `TwoPointConv`: "A flag (or identifier) indicating that the play was a two-point conversion attempt"

- `ChalReplayResult`: "The outcome/result of a coach challenge / instant replay review on that play"

- `RecFumbPlayer`: "The player who recovered a fumble on a reception (or possibly during a catch attempt)"

Essentially, these data points handle very 'niche' cases; two-point conversions, play challenges, and reception fumbles occur very rarely, and as such, on plays where these conditions do not occur, their values = NaN.

### Handling problematic data
After identifying the problematic data, interpreting these points is essential to the data cleaning phase. One method (and the easiest method) of handling missing data is to simply drop the rows associated with the data points. We can do this as shown:

In [19]:
# dropping all rows with any missing values
data_dropNA = data.dropna()
data_dropNA.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


In [20]:
print(f"Original data shape: {data.shape}")
print(f"Data shape after dropping rows with any missing values: {data_dropNA.shape}")

Original data shape: (407688, 102)
Data shape after dropping rows with any missing values: (0, 102)


The ".shape()" function returns the dataframe in the format "(n, m)", where:
- n = number of rows
- m = number of columns (features)
As you can see, dropping all rows containing an NaN drops **all** the rows in our dataset; we no longer have any data to work with, so this is an incorrect approach for this case.

An alternative option is to drop all the niche columns. For this workshop, we can consider this as "all the columns containing an NaN value".

In [21]:
# dropping all columns with any missing values
data_dropNA_cols = data.dropna(axis=1)
data_dropNA_cols.head()

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,...,AwayTeam,Timeout_Indicator,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob,Season
0,2009-09-10,2009091000,1,1,15,0,0,1,39,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
1,2009-09-10,2009091000,1,1,15,10,5,1,5,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
2,2009-09-10,2009091000,1,1,15,5,2,1,-3,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
3,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
4,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009


In [22]:
print(f"Original data shape: {data.shape}")
print(f"Data shape after dropping columns with any missing values: {data_dropNA_cols.shape}")

Original data shape: (407688, 102)
Data shape after dropping columns with any missing values: (407688, 37)


This worked much better! We have retained all the rows of our data, and have instead just dropped our NaN columns. We can do another check to see the amount of NaN values in our dataset:

In [16]:
print(data_dropNA_cols.isnull().sum())

Date                           0
GameID                         0
Drive                          0
qtr                            0
TimeUnder                      0
ydstogo                        0
ydsnet                         0
PlayAttempted                  0
Yards.Gained                   0
sp                             0
Touchdown                      0
Safety                         0
Onsidekick                     0
PlayType                       0
PassAttempt                    0
AirYards                       0
YardsAfterCatch                0
QBHit                          0
InterceptionThrown             0
RushAttempt                    0
Reception                      0
Fumble                         0
Sack                           0
Challenge.Replay               0
Accepted.Penalty               0
Penalty.Yards                  0
HomeTeam                       0
AwayTeam                       0
Timeout_Indicator              0
posteam_timeouts_pre           0
HomeTimeou

Simply removing NaN values is an easy solution to missing data, however, depending on the dataset, you may want to consider another solution, such as:

- replacing NaN with the average value for that feature
- defaulting to a specific value

**Let's take a look at these alternative solutions below:**

In [32]:
# getting a subset of the dataset used above
alt_data = data.loc[:, 'EPA':'Season'].head()
alt_data 

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,NaN,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,NaN,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,NaN,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


Above, you can see there are NaN values in multiple features: `airEPA`, `yacEPA`, `airWPA`, `yacWPA`. Again, we can interpret these features:

- `airEPA`: "Air Expected Points Added - Measures how much a play’s through-the-air component (the pass before the catch) contributes to the team’s expected points."

- `yacEPA`: "Yards After Catch Expected Points Added - Measures how much the receiver’s yards after the catch contribute to the expected points."

- `airEPA`: "Air Win Probability Added - The change in win probability attributable to the air yards portion of a pass play."

- `yacWPA`: "Yards After Catch Win Probability Added - The change in win probability resulting from the YAC portion of a pass play."

The 'NaN' values in this sub-dataset can be explained by the context - the value is 'NaN' for these 4 features when the play does not involve a throw; the play was a run, kick, or incomplete.

As such, filling these values with '0' could be a valid option; these features provide 0 difference, and as such, have no impact for these values.

In [31]:
# filling NaN values with 0
fill_cols = ['airEPA', 'yacEPA', 'airWPA', 'yacWPA']
alt_data[fill_cols] = alt_data[fill_cols].fillna(0)
alt_data.head()

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,0.000000,0.000000,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.000000,0.000000,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,0.000000,0.000000,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.000000,0.000000,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


---- 

Our data is now cleaned (again), and can be used for whatever our needs are. As data scientists, or individuals handling data, data cleaning is often the most important stage in the data lifecycle. Preparing data effectively will make visualization, modelling, and other processes much smoother.

Keep in mind that sometimes filling with 0 or dropping data isn't appropriate. Besides dropping data, there is not really any "one-size-fits-all" solution to missing/empty or NaN/null data. The method we implement is highly dependent on the context, the importance of the data, if the data is numeric or categorical, and if the data is discrete or continuous.

**Thank you for attending the first ever DataJam 2025 workshop, and we look forward to seeing familiar faces at the upcoming "Data Visualization with Python" and "Git + Github" workshops, as well as "DataJam 2025" on Saturday. Have a great evening!**